In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from pydataset import data

Paper by: Hadley Wickham

https://vita.had.co.nz/papers/tidy-data.pdf

### Data cleaning:

tidy data != clean data


- outlier checking  
- date parsing
- missing value imputation etc.
- <ins>data tidying: structuring datasets to facilitate analysis.</ins>



### Data semantics: tidy data

- Value: every value belongs to a variable and an observation.
- Variable: a variable contains all values that measure the same underlying attribute (like height, temperature, duration) across units.
- Observation: an observation contains all values measured on the same unit (like a person, or a day, or a race) across attributes.

####  datasets for lesson and exercises:

- https://classroom.google.com/c/MzUxODYzODI3NTE5/m/MzUxODYzODI3Njg5/details

In [5]:
# lets look at this data:
treatments = pd.read_csv('treatment.csv')
treatments

,Unnamed: 0,treatmenta,treatmentb,treatmentc
0,John Smith,NaN,2,0
1,Jane Doe,16.0,11,3
2,Mary Johnson,3.0,1,4


In [6]:
#rename columns
treatments.columns = ['name', 'a', 'b', 'c']
treatments

,name,a,b,c
0,John Smith,NaN,2,0
1,Jane Doe,16.0,11,3
2,Mary Johnson,3.0,1,4


What is an observation here?  
variables?  
values?  

In [7]:
# restructure data using 'melt'

treatments = treatments.melt(id_vars=['name'], 
                             var_name='treatment', 
                             value_name='response')

In [8]:
treatments

,name,treatment,response
0,John Smith,a,NaN
1,Jane Doe,a,16.0
2,Mary Johnson,a,3.0
3,John Smith,b,2.0
4,Jane Doe,b,11.0
5,Mary Johnson,b,1.0
6,John Smith,c,0.0
7,Jane Doe,c,3.0
8,Mary Johnson,c,4.0


### Tidy data : 
- Each variable forms a column.
- Each observation forms a row.
- Each cell has a single value.
- data is tabular, i.e. made up of rows and columns

In [9]:
# Examples of tidy-data?

data('tips')

,total_bill,tip,sex,smoker,day,time,size
1,16.99,1.01,Female,No,Sun,Dinner,2
2,10.34,1.66,Male,No,Sun,Dinner,3
3,21.01,3.50,Male,No,Sun,Dinner,3
4,23.68,3.31,Male,No,Sun,Dinner,2
5,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
240,29.03,5.92,Male,No,Sat,Dinner,3
241,27.18,2.00,Female,Yes,Sat,Dinner,2
242,22.67,2.00,Male,Yes,Sat,Dinner,2
243,17.82,1.75,Male,No,Sat,Dinner,2


#### General Ideas  
- If the units are the same, maybe they should be in the same column.
- If one column has measurements of different units, it should be spread out  
- Should you be able to groupby some of the columns? combine them  
- Can I pass this data to seaborn?  

- Can we ask interesting questions and answer them with a groupby? i.e. generally we don't want to be taking row or column averages.


### How to deal with 'messy' data


##### Reshaping data:  
  
- Wide data --> Long data format (Melt)  
- Long data --> Wide Data format (pivot_table, unstack)  

#### 1. Messy data: Column headers are values, not variable names.

In [10]:
df = pd.read_csv('pew.csv')
df.head()

,religion,<$10k,$10-20k,$20-30k,$30-40k,$40-50k,$50-75k,$75-100k,$100-150k,>150k,Don't know/refused
0,Agnostic,27,34,60,81,76,137,122,109,84,96
1,Atheist,12,27,37,52,35,70,73,59,74,76
2,Buddhist,27,21,30,34,33,58,62,39,53,54
3,Catholic,418,617,732,670,638,1116,949,792,633,1489
4,Don’t know/refused,15,14,15,11,10,35,21,17,18,116


In [11]:
# look at info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   religion            18 non-null     object
 1   <$10k               18 non-null     int64 
 2   $10-20k             18 non-null     int64 
 3   $20-30k             18 non-null     int64 
 4   $30-40k             18 non-null     int64 
 5   $40-50k             18 non-null     int64 
 6   $50-75k             18 non-null     int64 
 7   $75-100k            18 non-null     int64 
 8   $100-150k           18 non-null     int64 
 9   >150k               18 non-null     int64 
 10  Don't know/refused  18 non-null     int64 
dtypes: int64(10), object(1)
memory usage: 1.7+ KB


In [12]:
df.religion.value_counts()

Agnostic                   1
Atheist                    1
Other World Religions      1
Other Faiths               1
Other Christian            1
Orthodox                   1
Muslim                     1
Mormon                     1
Mainline Prot              1
Jewish                     1
Jehovah's Witness          1
Historically Black Prot    1
Hindu                      1
Evangelical Prot           1
Don’t know/refused         1
Catholic                   1
Buddhist                   1
Unaffiliated               1
Name: religion, dtype: int64

In [13]:
# Melt the data. Creates two new columns with default name of 'variable' and 'value'

df.melt(id_vars = 'religion')

,religion,variable,value
0,Agnostic,<$10k,27
1,Atheist,<$10k,12
2,Buddhist,<$10k,27
3,Catholic,<$10k,418
4,Don’t know/refused,<$10k,15
...,...,...,...
175,Orthodox,Don't know/refused,73
176,Other Christian,Don't know/refused,18
177,Other Faiths,Don't know/refused,71
178,Other World Religions,Don't know/refused,8


In [14]:
# melt data and specify names of new columns

df_tidy = df.melt(id_vars = 'religion', 
                  var_name = 'income', 
                  value_name = 'count')
df_tidy

,religion,income,count
0,Agnostic,<$10k,27
1,Atheist,<$10k,12
2,Buddhist,<$10k,27
3,Catholic,<$10k,418
4,Don’t know/refused,<$10k,15
...,...,...,...
175,Orthodox,Don't know/refused,73
176,Other Christian,Don't know/refused,18
177,Other Faiths,Don't know/refused,71
178,Other World Religions,Don't know/refused,8


#### pd.melt arguments
- id_vars = columns you want to keep (not melt)
- var_name = name of new column you created by melting columns
- value_name = column name for values

#### Another example: one variable stored across multiple columns

In [15]:
billboard = pd.read_csv('billboard.csv', encoding= 'unicode_escape')

In [16]:
billboard.head()

,year,artist,track,time,genre,date.entered,date.peaked,x1st.week,x2nd.week,x3rd.week,...,x67th.week,x68th.week,x69th.week,x70th.week,x71st.week,x72nd.week,x73rd.week,x74th.week,x75th.week,x76th.week
0,2000,Destiny's Child,Independent Women Part I,3:38,Rock,9/23/00,11/18/00,78,63.0,49.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000,Santana,"Maria, Maria",4:18,Rock,2/12/00,4/8/00,15,8.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000,Savage Garden,I Knew I Loved You,4:07,Rock,10/23/99,1/29/00,71,48.0,43.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000,Madonna,Music,3:45,Rock,8/12/00,9/16/00,41,23.0,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,8/5/00,10/14/00,57,47.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
billboard.shape

(317, 83)

In [18]:
# what is mean rating for each track?
billboard.iloc[:,7:].mean(axis = 1)

0      14.821429
1      10.500000
2      17.363636
3      13.458333
4      19.952381
         ...    
312    98.000000
313    99.000000
314    99.000000
315    99.000000
316    99.000000
Length: 317, dtype: float64

In [19]:
billboard.columns[:7]

Index(['year', 'artist', 'track', 'time', 'genre', 'date.entered',
       'date.peaked'],
      dtype='object')

In [22]:
# melt the data

billboard_melt = billboard.melt(id_vars = billboard.columns[:7],
                               var_name = 'week',
                               value_name = 'rating')

billboard_melt.head()

,year,artist,track,time,genre,date.entered,date.peaked,week,rating
0,2000,Destiny's Child,Independent Women Part I,3:38,Rock,9/23/00,11/18/00,x1st.week,78.0
1,2000,Santana,"Maria, Maria",4:18,Rock,2/12/00,4/8/00,x1st.week,15.0
2,2000,Savage Garden,I Knew I Loved You,4:07,Rock,10/23/99,1/29/00,x1st.week,71.0
3,2000,Madonna,Music,3:45,Rock,8/12/00,9/16/00,x1st.week,41.0
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,8/5/00,10/14/00,x1st.week,57.0


In [25]:
billboard_melt.shape

(24092, 9)

In [26]:
# what is the mean rating (across all weeks) of each track?
billboard_melt.groupby('track').rating.mean()

track
(Hot S**t) Country Grammar      30.882353
3 Little Words                  94.444444
911                             60.000000
A Country Boy Can Survive       86.666667
A Little Gasoline               89.833333
                                  ...    
You Won't Be Lonely Now         88.692308
You'll Always Be Loved By Me    76.842105
You're A God                    37.000000
Your Everything                 72.000000
www.memory                      63.400000
Name: rating, Length: 316, dtype: float64

#### 2. Messy data: Multiple variables are stored in one column.

In [ ]:
df = pd.DataFrame({
    'name': ['Sally', 'Jane', 'Billy', 'Suzy'],
    'pet': ['dog: max', 'dog: buddy', 'cat: grizabella', 'hamster: fred']
})
df

In [ ]:
# how to split the string


In [ ]:
# how to split a pandas column/series


In [ ]:
# Another (better) way


In [ ]:
# create two new columns


In [ ]:
df.head()

#### Messy data: Variables are stored in both rows and columns

In [ ]:
weather = pd.read_csv('untidy-data/weather.csv')

weather.head()

In [ ]:
# how many rows and columns?
weather.shape

In [ ]:
# melt the 'days'

weather_long = weather.melt(id_vars = ['id', 'year', 'month', 'element'],
                           var_name = 'day',
                           value_name = 'temp')

weather_long.head()

In [ ]:
weather_long.shape

In [ ]:
# pivot the element column. Reset index to go from multi-index to flat dataframe.

# weather_tidy = 

#### pd.pivot_table arguments
- Index = columns you want to keep (not pivot)
- columns = column you want to pivot
- values = values we want to populate in the new columns
- aggfunct = how you want to aggregate the duplicate rows

## Mini Exercise:

- read in excel file named dem_score.xls
- convert the data in tidy format (hint: melt the data)
- convert the melted dataframe back in wide-format (hint: pivot the data)




In [ ]:
# read in the excel file named dem_score.xlsx
df = pd.read_excel('untidy-data/dem_score.xlsx')
df.head()

In [ ]:
# is the data in tidy format? What is the shape of data

df.shape

In [ ]:
# convert dataframe in 'tidy' format (hint: pd.melt)

df_long = pd.melt(df, id_vars = 'country',
                   var_name = 'year',
                   value_name = 'score')

df_long.head()

In [ ]:
# check shape of tidy dataframe
df_long.shape

In [ ]:
# convert the data back in wide format (hint: pivot_table)
df_long.pivot_table(index = 'country',
                   columns = 'year',
                   values = 'score').reset_index()


#### Bit more complex example

In [ ]:
sales = pd.read_csv('untidy-data/sales.csv')

In [ ]:
#check head
sales.head()

In [ ]:
#check the shape
sales.shape

In [ ]:
# first we melt all the columns expect 'Product'

sales_melt[['year', 'measure']] = sales_melt.variable.str.split(' ', expand = True)

In [ ]:
# shape of melted dataset


In [ ]:
# here we split the 'variable' column to create two new columns 'year' and 'measure'
sales_melt[['year',]]

In [ ]:
# now we will pivot the 'measure' column to create the two new columns 'PPU' and 'Sales'
# reset the index to flatten the dataframe 

# sales_tidy 